In [1]:
# =============================================================
# Load Overlay & Map Hardware
# =============================================================
from pynq import Overlay, MMIO
import time
import numpy as np

BITSTREAM = "design_1_wrapper.bit"  #

print("Loading Overlay...")
overlay = Overlay(BITSTREAM)
print("Overlay Loaded.")

print("IPs:", list(overlay.ip_dict.keys()))
print("Mem:", list(overlay.mem_dict.keys()))

# BRAM controllers are in mem_dict
iram_info = overlay.mem_dict['axi_bram_ctrl_1']
dram_info = overlay.mem_dict['axi_bram_ctrl_0']
iram = MMIO(iram_info['phys_addr'], iram_info['addr_range'])
dram = MMIO(dram_info['phys_addr'], dram_info['addr_range'])

# GPIO is in ip_dict
gpio_info = overlay.ip_dict['axi_gpio_0']
gpio = MMIO(gpio_info['phys_addr'], gpio_info['addr_range'])

def halt_core():
    gpio.write(0x0, 0x0)

def run_core():
    gpio.write(0x0, 0x1)

# for debugging
print(f"IRAM: {iram_info['addr_range']//4} words, DRAM: {dram_info['addr_range']//4} words")
print("Hardware mapped.")

Loading Overlay...


Overlay Loaded.
IPs: ['axi_gpio_0', 'processing_system7_0']
Mem: ['axi_bram_ctrl_0', 'axi_bram_ctrl_1', 'PSDDR']
IRAM: 2048 words, DRAM: 2048 words
Hardware mapped.


In [2]:
# =============================================================
# Memory Sanity Check
# =============================================================
print("=" * 50)
print("SANITY CHECK")
print("=" * 50)
halt_core()
errors = 0

for pattern in [0xDEADBEEF, 0x01020304, 0xFFFFFFFF, 0x00000000]:
    iram.write(0x0, pattern)
    rb = iram.read(0x0)
    ok = rb == pattern
    if not ok: errors += 1
    print(f"  IRAM 0x{pattern:08X} -> 0x{rb:08X} [{'OK' if ok else 'FAIL'}]")

for pattern in [0xCAFEBABE, 0x12345678, 0x00000000]:
    dram.write(0x0, pattern)
    rb = dram.read(0x0)
    ok = rb == pattern
    if not ok: errors += 1
    print(f"  DRAM 0x{pattern:08X} -> 0x{rb:08X} [{'OK' if ok else 'FAIL'}]")

# Test multiple offsets
for off in [0x0, 0x4, 0x10, 0x7C, 0x100]:
    dram.write(off, 0xAA000000 | off)
    rb = dram.read(off)
    ok = rb == (0xAA000000 | off)
    if not ok: errors += 1
    print(f"  DRAM[0x{off:03X}] = 0x{rb:08X} [{'OK' if ok else 'FAIL'}]")

gpio.write(0x0, 0x0)
v0 = gpio.read(0x0) & 0x1
gpio.write(0x0, 0x1)
v1 = gpio.read(0x0) & 0x1
gpio_ok = (v0 == 0 and v1 == 1)
if not gpio_ok: errors += 1
print(f"  GPIO: 0->{v0}, 1->{v1} [{'OK' if gpio_ok else 'FAIL'}]")
halt_core()

print("=" * 50)
print("ALL PASSED" if errors == 0 else f"FAILED: {errors} error(s)")
print("=" * 50)

SANITY CHECK
  IRAM 0xDEADBEEF -> 0xDEADBEEF [OK]
  IRAM 0x01020304 -> 0x01020304 [OK]
  IRAM 0xFFFFFFFF -> 0xFFFFFFFF [OK]
  IRAM 0x00000000 -> 0x00000000 [OK]
  DRAM 0xCAFEBABE -> 0xCAFEBABE [OK]
  DRAM 0x12345678 -> 0x12345678 [OK]
  DRAM 0x00000000 -> 0x00000000 [OK]
  DRAM[0x000] = 0xAA000000 [OK]
  DRAM[0x004] = 0xAA000004 [OK]
  DRAM[0x010] = 0xAA000010 [OK]
  DRAM[0x07C] = 0xAA00007C [OK]
  DRAM[0x100] = 0xAA000100 [OK]
  GPIO: 0->0, 1->1 [OK]
ALL PASSED


In [3]:
# =============================================================
# Smoke Test (check if core showss lifesigns)
# =============================================================
print("=" * 50)
print("SMOKE TEST")
print("=" * 50)
halt_core()

# Clear memories
for i in range(16):
    iram.write(i * 4, 0x00000013)  # NOP
    dram.write(i * 4, 0xFFFFFFFF)  # sentinel

# addi x1,x0,42; sw x1,0(x0); addi x2,x0,7; sw x2,4(x0); beq x0,x0,0
smoke = [0x02A00093, 0x00102023, 0x00700113, 0x00202223, 0x00000063]
for i, instr in enumerate(smoke):
    iram.write(i * 4, instr)

run_core()
time.sleep(0.5)
halt_core()

v0 = dram.read(0x0)
v1 = dram.read(0x4)
print(f"DRAM[0] = {v0} (expect 42), DRAM[1] = {v1} (expect 7)")
if v0 == 42 and v1 == 7:
    print(">>> SMOKE TEST PASSED <<<")
else:
    print(">>> SMOKE TEST FAILED <<<")

SMOKE TEST
DRAM[0] = 42 (expect 42), DRAM[1] = 7 (expect 7)
>>> SMOKE TEST PASSED <<<


In [4]:
# ================================================================
# Addition Test
# ================================================================
print("=" * 50)
print("ADDITION TEST")
print("=" * 50)
halt_core()

# Clear memories
for i in range(16):
    iram.write(i * 4, 0x00000013)  # NOP
    dram.write(i * 4, 0xFFFFFFFF)  # sentinel

# addi x1, x0, 5       -> x1 = 5
# addi x2, x0, 10      -> x2 = 10
# add  x3, x1, x2      -> x3 = 15
# sw   x3, 8(x0)       -> store result to mem[8]
# beq  x0, x0, 0       -> halt (infinite loop)
addition = [
    0x00500093,  # addi x1, x0, 5
    0x00A00113,  # addi x2, x0, 10
    0x002081B3,  # add  x3, x1, x2
    0x00302423,  # sw   x3, 8(x0)
    0x00000063,  # beq  x0, x0, 0  (halt)
]
for i, instr in enumerate(addition):
    iram.write(i * 4, instr)

run_core()
time.sleep(0.5)
halt_core()

v2 = dram.read(0x8)
print(f"DRAM[8] = {v2} (expect 15)")
if v2 == 15:
    print(">>> ADDITION TEST PASSED <<<")
else:
    print(">>> ADDITION TEST FAILED <<<")

ADDITION TEST
DRAM[8] = 15 (expect 15)
>>> ADDITION TEST PASSED <<<


In [5]:
# ================================================================
# Addition Test 2 (different numbers)
# ================================================================
print("=" * 50)
print("ADDITION TEST 2")
print("=" * 50)
halt_core()

# Clear memories
for i in range(16):
    iram.write(i * 4, 0x00000013)  # NOP
    dram.write(i * 4, 0xFFFFFFFF)  # sentinel

# addi x1, x0, 25      -> x1 = 25
# addi x2, x0, 37      -> x2 = 37
# add  x3, x1, x2      -> x3 = 62
# sw   x3, 8(x0)       -> store result to mem[8]
# beq  x0, x0, 0       -> halt
addition2 = [
    0x01900093,  # addi x1, x0, 25
    0x02500113,  # addi x2, x0, 37
    0x002081B3,  # add  x3, x1, x2
    0x00302423,  # sw   x3, 8(x0)
    0x00000063,  # beq  x0, x0, 0  (halt)
]
for i, instr in enumerate(addition2):
    iram.write(i * 4, instr)

run_core()
time.sleep(0.5)
halt_core()

v2 = dram.read(0x8)
print(f"DRAM[8] = {v2} (expect 62)")
if v2 == 62:
    print(">>> ADDITION TEST 2 PASSED <<<")
else:
    print(">>> ADDITION TEST 2 FAILED <<<")

ADDITION TEST 2
DRAM[8] = 62 (expect 62)
>>> ADDITION TEST 2 PASSED <<<


In [6]:
# ================================================================
# SLT Signed Comparison Test (pre-sort sanity check)
# ================================================================
print("=" * 50)
print("SIGNED COMPARISON TEST")
print("=" * 50)
halt_core()

for i in range(16):
    iram.write(i * 4, 0x00000013)
    dram.write(i * 4, 0xFFFFFFFF)

# addi x1, x0, -5      -> x1 = -5  (0xFFFFFFFB)
# addi x2, x0, 3       -> x2 = 3
# slt  x3, x1, x2      -> x3 should = 1 (-5 < 3)
# slt  x4, x2, x1      -> x4 should = 0 (3 < -5 is false)
# sw   x3, 0(x0)       -> DRAM[0] = 1
# sw   x4, 4(x0)       -> DRAM[4] = 0
# beq  x0, x0, 0       -> halt
slt_test = [
    0xFFB00093,  # addi x1, x0, -5
    0x00300113,  # addi x2, x0, 3
    0x0020A1B3,  # slt  x3, x1, x2
    0x0010A233,  # slt  x4, x2, x1
    0x00302023,  # sw   x3, 0(x0)
    0x00402223,  # sw   x4, 4(x0)
    0x00000063,  # beq  x0, x0, 0
]
for i, instr in enumerate(slt_test):
    iram.write(i * 4, instr)

run_core()
time.sleep(0.5)
halt_core()

v0 = dram.read(0x0)
v1 = dram.read(0x4)
print(f"slt(-5, 3) = {v0} (expect 1)")
print(f"slt(3, -5) = {v1} (expect 0)")
if v0 == 1 and v1 == 0:
    print(">>> SIGNED COMPARISON TEST PASSED <<<")
else:
    print(">>> SIGNED COMPARISON TEST FAILED - this explains sort failure! <<<")

SIGNED COMPARISON TEST
slt(-5, 3) = 1 (expect 1)
slt(3, -5) = 0 (expect 0)
>>> SIGNED COMPARISON TEST PASSED <<<


In [7]:
# ================================================================
# Simple Bubble Sort Test (8 elements, all positive)
# ================================================================
print("=" * 50)
print("BUBBLE SORT TEST (8 elements)")
print("=" * 50)
halt_core()

# Clear memories
for i in range(32):
    iram.write(i * 4, 0x00000013)  # NOP
    dram.write(i * 4, 0xFFFFFFFF)  # sentinel

# Test data - small positive numbers, easy to verify
test_data = [5, 3, 8, 1, 9, 2, 7, 4]
golden = sorted(test_data)  # [1, 2, 3, 4, 5, 7, 8, 9]
print(f"Input:    {test_data}")
print(f"Expected: {golden}")

# Load test data into DRAM
for i, val in enumerate(test_data):
    dram.write(i * 4, val)

# Register usage:
#   x1 = i (outer counter, 0..N-2)
#   x2 = N-1 = 7
#   x3 = inner limit (N-1-i)
#   x4 = j (inner counter)
#   x5 = byte offset (j*4)
#   x6 = arr[j]
#   x7 = arr[j+1]
#   x8 = slt result
#
program = [
    # 0: 0x00  addi x1, x0, 0       i = 0
    0x00000093,
    # 1: 0x04  addi x2, x0, 7       N-1 = 7
    0x00700113,

    # OUTER LOOP
    # 2: 0x08  sub x3, x2, x1       limit = N-1-i
    0x401101B3,
    # 3: 0x0C  slt x8, x1, x2       i < N-1?
    0x0020A433,
    # 4: 0x10  beq x8, x0, +56      goto DONE instr 18 (0x48): offset=0x48-0x10=+56=0x38
    #          imm=56: [12]=0,[11]=0,[10:5]=011100,[4:1]=0000
    #          0_011100_00000_01000_000_0000_0_1100011
    0x03840463,

    # 5: 0x14  addi x4, x0, 0       j = 0
    0x00000213,

    # INNER LOOP
    # 6: 0x18  slt x8, x4, x3       j < limit?
    0x00322433,
    # 7: 0x1C  beq x8, x0, +32      goto OUTER_INC instr 15 (0x3C): offset=0x3C-0x1C=+32=0x20
    #          imm=32: [12]=0,[11]=0,[10:5]=010000,[4:1]=0000
    #          0_010000_00000_01000_000_0000_0_1100011
    0x02040063,

    # 8: 0x20  slli x5, x4, 2       x5 = j*4
    0x00221293,
    # 9: 0x24  lw x6, 0(x5)         x6 = arr[j]
    0x00028303,
    # 10: 0x28  lw x7, 4(x5)        x7 = arr[j+1]
    0x00428383,
    # 11: 0x2C  slt x8, x7, x6      arr[j+1] < arr[j]?
    0x0063A433,
    # 12: 0x30  beq x8, x0, +12     no swap, goto instr 15 (0x3C): offset=0x3C-0x30=+12
    #          imm=12: [12]=0,[11]=0,[10:5]=000110,[4:1]=0000  -- wait, 12=0b1100
    #          imm[4:1]=0110, imm[10:5]=000000
    #          0_000000_00000_01000_000_0110_0_1100011
    0x00C40663,
    # 13: 0x34  sw x7, 0(x5)        arr[j] = arr[j+1]
    0x00728023,
    # 14: 0x38  sw x6, 4(x5)        arr[j+1] = arr[j]
    0x00628223,

    # INC_J
    # 15: 0x3C  addi x4, x4, 1      j++
    0x00120213,
    # 16: 0x40  beq x0, x0, -40     goto INNER LOOP instr 6 (0x18): offset=0x18-0x40=-40
    #          -40 = 0xFFFFFFD8, binary: 1_111110_11000  -- let me be careful
    #          -40 in 13-bit signed: 1_1111_1101_1000
    #          imm[12]=1, imm[11]=1, imm[10:5]=111110, imm[4:1]=1100
    #          1_111110_00000_00000_000_1100_1_1100011
    0xFE000CE3,  # CORRECTED from 0xFD800CE3

    # OUTER_INC
    # 17: 0x44  addi x1, x1, 1      i++
    0x00108093,
    # 18: 0x48  beq x0, x0, -64     goto OUTER LOOP instr 2 (0x08): offset=0x08-0x48=-64
    #          -64 = 0xFFFFFFC0
    #          imm[12]=1, imm[11]=1, imm[10:5]=111100, imm[4:1]=0000
    #          1_111100_00000_00000_000_0000_1_1100011
    0xFC000EE3,

    # DONE
    # 19: 0x4C  addi x9, x0, 1      done = 1
    0x00100493,
    # 20: 0x50  sw x9, 128(x0)      DRAM[0x80] = 1
    #          sw: imm[11:5]=0000100, rs2=x9, rs1=x0, imm[4:0]=00000
    #          0000100_01001_00000_010_00000_0100011
    0x00902023,  # offset 128 -- actually let me verify: 128=0x80
                 # imm[11:5] = 0000100, imm[4:0] = 00000
                 # 0000100_01001_00000_010_00000_0100011 = 0x08902023... hmm
    # 21: 0x54  beq x0, x0, 0       halt
    0x00000063,
]
# Load program
for i, instr in enumerate(program):
    iram.write(i * 4, instr)
print(f"Loaded {len(program)} instructions")

# Verify IRAM
ok = True
for i, instr in enumerate(program):
    v = iram.read(i * 4)
    if v != instr:
        print(f"  IRAM[{i}] MISMATCH: wrote 0x{instr:08X}, read 0x{v:08X}")
        ok = False
if ok:
    print("IRAM verified OK")

# Run
run_core()
time.sleep(1.0)  # give it time to sort
halt_core()

# Check done flag
done_val = dram.read(0x80)  # address 128
print(f"Done flag at DRAM[128] = {done_val} (expect 1)")

# Read result
result = [dram.read(i * 4) for i in range(8)]
print(f"Output:   {result}")
print(f"Expected: {golden}")

errors = sum(1 for i in range(8) if result[i] != golden[i])
if errors == 0 and done_val == 1:
    print(">>> BUBBLE SORT TEST PASSED <<<")
else:
    print(f">>> BUBBLE SORT TEST FAILED ({errors} mismatches) <<<")

BUBBLE SORT TEST (8 elements)
Input:    [5, 3, 8, 1, 9, 2, 7, 4]
Expected: [1, 2, 3, 4, 5, 7, 8, 9]
Loaded 22 instructions
IRAM verified OK
Done flag at DRAM[128] = 0 (expect 1)
Output:   [5, 5, 8, 1, 9, 2, 7, 4]
Expected: [1, 2, 3, 4, 5, 7, 8, 9]
>>> BUBBLE SORT TEST FAILED (8 mismatches) <<<


In [8]:
# Quick diagnostic - print what's actually in IRAM
print("=== IRAM DIAGNOSTIC ===")
for i in range(22):
    v = iram.read(i * 4)
    print(f"  IRAM[{i:2d}] @ 0x{i*4:02X} = 0x{v:08X}")

=== IRAM DIAGNOSTIC ===
  IRAM[ 0] @ 0x00 = 0x00000093
  IRAM[ 1] @ 0x04 = 0x00700113
  IRAM[ 2] @ 0x08 = 0x401101B3
  IRAM[ 3] @ 0x0C = 0x0020A433
  IRAM[ 4] @ 0x10 = 0x03840463
  IRAM[ 5] @ 0x14 = 0x00000213
  IRAM[ 6] @ 0x18 = 0x00322433
  IRAM[ 7] @ 0x1C = 0x02040063
  IRAM[ 8] @ 0x20 = 0x00221293
  IRAM[ 9] @ 0x24 = 0x00028303
  IRAM[10] @ 0x28 = 0x00428383
  IRAM[11] @ 0x2C = 0x0063A433
  IRAM[12] @ 0x30 = 0x00C40663
  IRAM[13] @ 0x34 = 0x00728023
  IRAM[14] @ 0x38 = 0x00628223
  IRAM[15] @ 0x3C = 0x00120213
  IRAM[16] @ 0x40 = 0xFE000CE3
  IRAM[17] @ 0x44 = 0x00108093
  IRAM[18] @ 0x48 = 0xFC000EE3
  IRAM[19] @ 0x4C = 0x00100493
  IRAM[20] @ 0x50 = 0x00902023
  IRAM[21] @ 0x54 = 0x00000063


In [9]:
# Branch offset verifier
def encode_beq(rs1, rs2, offset):
    """Encode BEQ instruction and verify offset"""
    assert offset % 2 == 0, "offset must be even"
    assert -4096 <= offset <= 4094, "offset out of range"
    imm = offset & 0x1FFF  # 13-bit signed
    imm12  = (imm >> 12) & 0x1
    imm11  = (imm >> 11) & 0x1
    imm10_5 = (imm >> 5)  & 0x3F
    imm4_1  = (imm >> 1)  & 0xF
    instr = (imm12  << 31) | \
            (imm10_5 << 25) | \
            (rs2    << 20) | \
            (rs1    << 15) | \
            (0b000  << 12) | \
            (imm4_1 << 8)  | \
            (imm11  << 7)  | \
            0b1100011
    return instr & 0xFFFFFFFF

print("=== BRANCH ENCODINGS ===")
print(f"instr  4: beq x8,x0,+56  = 0x{encode_beq(8,0,+56):08X}  (expect 0x03840463)")
print(f"instr  7: beq x8,x0,+32  = 0x{encode_beq(8,0,+32):08X}  (expect 0x02040063)")
print(f"instr 12: beq x8,x0,+12  = 0x{encode_beq(8,0,+12):08X}  (expect 0x00C40663)")
print(f"instr 16: beq x0,x0,-40  = 0x{encode_beq(0,0,-40):08X}  (expect 0xFE000CE3)")
print(f"instr 18: beq x0,x0,-64  = 0x{encode_beq(0,0,-64):08X}  (expect 0xFC000EE3)")

=== BRANCH ENCODINGS ===
instr  4: beq x8,x0,+56  = 0x02040C63  (expect 0x03840463)
instr  7: beq x8,x0,+32  = 0x02040063  (expect 0x02040063)
instr 12: beq x8,x0,+12  = 0x00040663  (expect 0x00C40663)
instr 16: beq x0,x0,-40  = 0xFC000CE3  (expect 0xFE000CE3)
instr 18: beq x0,x0,-64  = 0xFC0000E3  (expect 0xFC000EE3)


In [10]:
def encode_beq(rs1, rs2, offset):
    assert offset % 2 == 0
    assert -4096 <= offset <= 4094
    imm = offset & 0x1FFF
    imm12   = (imm >> 12) & 0x1
    imm11   = (imm >> 11) & 0x1
    imm10_5 = (imm >> 5)  & 0x3F
    imm4_1  = (imm >> 1)  & 0xF
    instr = (imm12   << 31) | \
            (imm10_5 << 25) | \
            (rs2     << 20) | \
            (rs1     << 15) | \
            (0b000   << 12) | \
            (imm4_1  << 8)  | \
            (imm11   << 7)  | \
            0b1100011
    return instr & 0xFFFFFFFF

def encode_sw(rs2, rs1, offset):
    """sw rs2, offset(rs1)"""
    imm = offset & 0xFFF
    imm11_5 = (imm >> 5) & 0x7F
    imm4_0  = imm & 0x1F
    instr = (imm11_5 << 25) | \
            (rs2     << 20) | \
            (rs1     << 15) | \
            (0b010   << 12) | \
            (imm4_0  << 7)  | \
            0b0100011
    return instr & 0xFFFFFFFF

# Instruction addresses:
# 0:0x00  1:0x04  2:0x08  3:0x0C  4:0x10
# 5:0x14  6:0x18  7:0x1C  8:0x20  9:0x24
# 10:0x28 11:0x2C 12:0x30 13:0x34 14:0x38
# 15:0x3C 16:0x40 17:0x44 18:0x48 19:0x4C
# 20:0x50 21:0x54

B4  = encode_beq(8, 0, 0x48-0x10)  # instr 4  -> DONE  (instr 18)
B7  = encode_beq(8, 0, 0x3C-0x1C)  # instr 7  -> OUTER_INC (instr 15)
B12 = encode_beq(8, 0, 0x3C-0x30)  # instr 12 -> INC_J (instr 15)
B16 = encode_beq(0, 0, 0x18-0x40)  # instr 16 -> INNER LOOP (instr 6)
B18 = encode_beq(0, 0, 0x08-0x48)  # instr 18 -> OUTER LOOP (instr 2)
SW20 = encode_sw(9, 0, 128)         # instr 20: sw x9, 128(x0)

print("=== COMPUTED ENCODINGS ===")
print(f"B4  = 0x{B4:08X}")
print(f"B7  = 0x{B7:08X}")
print(f"B12 = 0x{B12:08X}")
print(f"B16 = 0x{B16:08X}")
print(f"B18 = 0x{B18:08X}")
print(f"SW20= 0x{SW20:08X}")

program = [
    0x00000093,  #  0: addi x1, x0, 0       i = 0
    0x00700113,  #  1: addi x2, x0, 7       N-1 = 7
    0x401101B3,  #  2: sub  x3, x2, x1      limit = N-1-i   OUTER LOOP
    0x0020A433,  #  3: slt  x8, x1, x2      i < N-1?
    B4,          #  4: beq  x8, x0, ->DONE
    0x00000213,  #  5: addi x4, x0, 0       j = 0
    0x00322433,  #  6: slt  x8, x4, x3      j < limit?      INNER LOOP
    B7,          #  7: beq  x8, x0, ->OUTER_INC
    0x00221293,  #  8: slli x5, x4, 2       x5 = j*4
    0x00028303,  #  9: lw   x6, 0(x5)       x6 = arr[j]
    0x00428383,  # 10: lw   x7, 4(x5)       x7 = arr[j+1]
    0x0063A433,  # 11: slt  x8, x7, x6      arr[j+1] < arr[j]?
    B12,         # 12: beq  x8, x0, ->INC_J
    0x00728023,  # 13: sw   x7, 0(x5)       arr[j] = arr[j+1]
    0x00628223,  # 14: sw   x6, 4(x5)       arr[j+1] = arr[j]
    0x00120213,  # 15: addi x4, x4, 1       j++             INC_J
    B16,         # 16: beq  x0, x0, ->INNER LOOP
    0x00108093,  # 17: addi x1, x1, 1       i++             OUTER_INC
    B18,         # 18: beq  x0, x0, ->OUTER LOOP
    0x00100493,  # 19: addi x9, x0, 1       done = 1        DONE
    SW20,        # 20: sw   x9, 128(x0)     DRAM[128] = 1
    0x00000063,  # 21: beq  x0, x0, 0       halt
]

# ================================================================
print("=" * 50)
print("BUBBLE SORT TEST (8 elements) - encoder-verified")
print("=" * 50)
halt_core()

test_data = [5, 3, 8, 1, 9, 2, 7, 4]
golden = sorted(test_data)
print(f"Input:    {test_data}")
print(f"Expected: {golden}")

# Clear memories
for i in range(32):
    iram.write(i * 4, 0x00000013)
    dram.write(i * 4, 0xFFFFFFFF)

# Load program
for i, instr in enumerate(program):
    iram.write(i * 4, instr)

# Verify
ok = True
for i, instr in enumerate(program):
    v = iram.read(i * 4)
    if v != instr:
        print(f"  IRAM[{i}] MISMATCH: wrote 0x{instr:08X}, read 0x{v:08X}")
        ok = False
if ok:
    print(f"IRAM verified OK ({len(program)} instructions)")

# Print final encoding for reference
print("\n=== FINAL IRAM ===")
for i in range(len(program)):
    print(f"  [{i:2d}] 0x{i*4:02X}: 0x{program[i]:08X}")

# Load data
for i, val in enumerate(test_data):
    dram.write(i * 4, val)
dram.write(128, 0)

# Run
run_core()
time.sleep(1.0)
halt_core()

# Check
done_val = dram.read(128)
result = [dram.read(i * 4) for i in range(8)]
print(f"\nDone flag = {done_val} (expect 1)")
print(f"Output:   {result}")
print(f"Expected: {golden}")

errors = sum(1 for i in range(8) if result[i] != golden[i])
if errors == 0 and done_val == 1:
    print(">>> BUBBLE SORT PASSED <<<")
else:
    print(f">>> BUBBLE SORT FAILED ({errors} mismatches, done={done_val}) <<<")

=== COMPUTED ENCODINGS ===
B4  = 0x02040C63
B7  = 0x02040063
B12 = 0x00040663
B16 = 0xFC000CE3
B18 = 0xFC0000E3
SW20= 0x08902023
BUBBLE SORT TEST (8 elements) - encoder-verified
Input:    [5, 3, 8, 1, 9, 2, 7, 4]
Expected: [1, 2, 3, 4, 5, 7, 8, 9]
IRAM verified OK (22 instructions)

=== FINAL IRAM ===
  [ 0] 0x00: 0x00000093
  [ 1] 0x04: 0x00700113
  [ 2] 0x08: 0x401101B3
  [ 3] 0x0C: 0x0020A433
  [ 4] 0x10: 0x02040C63
  [ 5] 0x14: 0x00000213
  [ 6] 0x18: 0x00322433
  [ 7] 0x1C: 0x02040063
  [ 8] 0x20: 0x00221293
  [ 9] 0x24: 0x00028303
  [10] 0x28: 0x00428383
  [11] 0x2C: 0x0063A433
  [12] 0x30: 0x00040663
  [13] 0x34: 0x00728023
  [14] 0x38: 0x00628223
  [15] 0x3C: 0x00120213
  [16] 0x40: 0xFC000CE3
  [17] 0x44: 0x00108093
  [18] 0x48: 0xFC0000E3
  [19] 0x4C: 0x00100493
  [20] 0x50: 0x08902023
  [21] 0x54: 0x00000063

Done flag = 0 (expect 1)
Output:   [5, 5, 8, 1, 9, 2, 7, 4]
Expected: [1, 2, 3, 4, 5, 7, 8, 9]
>>> BUBBLE SORT FAILED (8 mismatches, done=0) <<<


In [11]:
# ================================================================
# Hazard Diagnostic Tests
# ================================================================

# Test 1: slli then immediate lw (RAW hazard on address)
print("=== TEST: slli -> lw hazard ===")
halt_core()
for i in range(16):
    iram.write(i * 4, 0x00000013)
    dram.write(i * 4, 0xFFFFFFFF)

# Put known value at address 8 (word 2)
dram.write(8, 0x000000AA)

# addi x4, x0, 2      x4 = 2
# slli x5, x4, 2      x5 = 8  (j*4 where j=2)
# lw   x6, 0(x5)      x6 = DRAM[8] = 0xAA
# sw   x6, 0(x0)      DRAM[0] = x6
# halt
test1 = [
    0x00200213,  # addi x4, x0, 2
    0x00221293,  # slli x5, x4, 2
    0x00028303,  # lw   x6, 0(x5)
    0x00602023,  # sw   x6, 0(x0)
    0x00000063,  # halt
]
for i, instr in enumerate(test1):
    iram.write(i * 4, instr)
run_core()
time.sleep(0.3)
halt_core()
v = dram.read(0)
print(f"DRAM[0] = 0x{v:08X} (expect 0x000000AA) {'PASS' if v == 0xAA else 'FAIL'}")

# Test 2: lw then immediate slt (load-use hazard)
print("=== TEST: lw -> slt hazard ===")
halt_core()
for i in range(16):
    iram.write(i * 4, 0x00000013)
    dram.write(i * 4, 0xFFFFFFFF)

dram.write(0, 5)   # arr[0] = 5
dram.write(4, 3)   # arr[1] = 3

# lw  x6, 0(x0)    x6 = 5
# lw  x7, 4(x0)    x7 = 3
# slt x8, x7, x6   x8 = (3 < 5) = 1
# sw  x8, 8(x0)    DRAM[8] = 1
# halt
test2 = [
    0x00002303,  # lw  x6, 0(x0)
    0x00402383,  # lw  x7, 4(x0)
    0x0063A433,  # slt x8, x7, x6
    0x00802423,  # sw  x8, 8(x0)
    0x00000063,  # halt
]
for i, instr in enumerate(test2):
    iram.write(i * 4, instr)
run_core()
time.sleep(0.3)
halt_core()
v = dram.read(8)
print(f"DRAM[8] = {v} (expect 1) {'PASS' if v == 1 else 'FAIL'}")

# Test 3: lw -> lw -> slt (two loads then compare)
print("=== TEST: lw -> lw -> slt (swap check) ===")
halt_core()
for i in range(16):
    iram.write(i * 4, 0x00000013)
    dram.write(i * 4, 0xFFFFFFFF)

dram.write(0, 5)
dram.write(4, 3)

# lw  x6, 0(x0)    x6 = 5
# lw  x7, 4(x0)    x7 = 3
# slt x8, x7, x6   x8 = 1 (swap needed)
# beq x8, x0, +8   skip swap if x8==0
# sw  x7, 0(x0)    DRAM[0] = 3
# sw  x6, 4(x0)    DRAM[4] = 5
# halt
test3 = [
    0x00002303,  # lw  x6, 0(x0)
    0x00402383,  # lw  x7, 4(x0)
    0x0063A433,  # slt x8, x7, x6
    encode_beq(8, 0, +8),  # beq x8, x0, +8
    0x00702023,  # sw  x7, 0(x0)
    0x00602223,  # sw  x6, 4(x0)
    0x00000063,  # halt
]
for i, instr in enumerate(test3):
    iram.write(i * 4, instr)
run_core()
time.sleep(0.3)
halt_core()
v0 = dram.read(0)
v1 = dram.read(4)
print(f"DRAM[0]={v0} (expect 3), DRAM[4]={v1} (expect 5) {'PASS' if v0==3 and v1==5 else 'FAIL'}")

=== TEST: slli -> lw hazard ===
DRAM[0] = 0x000000AA (expect 0x000000AA) PASS
=== TEST: lw -> slt hazard ===
DRAM[8] = 4294967295 (expect 1) FAIL
=== TEST: lw -> lw -> slt (swap check) ===
DRAM[0]=5 (expect 3), DRAM[4]=5 (expect 5) FAIL
